# 准备环境
需要下载相关依赖包，并设置API调用的环境变量。

不同大模型平台提供的环境变量不同，这里以百度千帆大模型平台的设置为例进行展示。
千帆平台上获取AK、SK的方法见文档 https://cloud.baidu.com/doc/Reference/s/9jwvz2egb

In [29]:
!pip list | grep langchain
!pip list | grep qianfan
!pip list | grep faiss-cpu

langchain                 0.2.1
langchain-community       0.2.1
langchain-core            0.2.3
langchain-text-splitters  0.2.0
qianfan                   0.3.14
faiss-cpu                 1.8.0


In [30]:
import os

os.environ['QIANFAN_AK'] = "xxx"
os.environ['QIANFAN_SK'] = "xxx"

# langchain的基本使用：调用大模型
LangChain（https://www.langchain.com/ ）是一个由语言模型LLM驱动的应用程序框架，它允许用户围绕大型语言模型快速构建应用程序。LangChain本身并不开发LLM，它的核心理念是为各种LLM提供通用的接口，降低开发者的学习成本，方便开发者快速地开发复杂的LLMs应用。它提供了一套工具、组件和接口，可简化创建由大型语言模型（LLM）和聊天模型提供支持的应用程序的过程。

本次课程将介绍如何用langchain来开发一个检索增强生成（RAG）应用。

首先看看langchain的基本使用，下面是用langchain来调用文心一言3.5 API的例子。

In [31]:
from langchain.chat_models import QianfanChatEndpoint

llm = QianfanChatEndpoint(model="ERNIE-3.5-8K")

In [32]:
llm.invoke("写一首学习RAG的诗")

AIMessage(content='学习RAG\n\nRAG语言妙无穷，逻辑编程显神通。\n数据结构精心构，函数方法巧运用。\n代码运行似流水，算法优化如飞鸿。\n勤奋学习终有成，编程之路任驰骋。', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-6twukzhetp', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 6, 'completion_tokens': 52, 'total_tokens': 58}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'normal', 'id': 'as-6twukzhetp', 'object': 'chat.completion', 'created': 1717344423, 'result': '学习RAG\n\nRAG语言妙无穷，逻辑编程显神通。\n数据结构精心构，函数方法巧运用。\n代码运行似流水，算法优化如飞鸿。\n勤奋学习终有成，编程之路任驰骋。', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 6, 'completion_tokens': 52, 'total_tokens': 58}}, id='run-9fc4027e-ecad-4e24-ad56-ebadd308e8a2-0')

可以在输入前加上一个问题模版（prompt template），让大模型回答问题时遵循模版上的要求。

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长写七言绝句的诗人，而且在诗中善于融入现代社会的工作学习方式。"),
    ("user", "{input}")
])

然后可以用langchain重载过的运算符 | 将问题模版和llm串联起来，组成一个调用链 chain。

In [33]:
chain = prompt | llm 

In [34]:
chain.invoke({"input": "写一首学习RAG的诗"})

AIMessage(content='学习RAG真奇妙，\n代码编织逻辑高。\n智能模型显神通，\n未来世界任逍遥。', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-fgg91yaeqt', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 29, 'completion_tokens': 24, 'total_tokens': 53}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'normal', 'id': 'as-fgg91yaeqt', 'object': 'chat.completion', 'created': 1717344553, 'result': '学习RAG真奇妙，\n代码编织逻辑高。\n智能模型显神通，\n未来世界任逍遥。', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 29, 'completion_tokens': 24, 'total_tokens': 53}}, id='run-dc63aeb1-8ee4-4035-80ae-0e3a9c9eda5a-0')

# 检索增强生成（RAG）方案
总体上分为准备阶段和调用阶段。

## 准备阶段
1. 准备待检索的文档，最好是清洗之后的纯文本格式
2. 将文档切分成文档片段
3. 选择向量模型和向量库，将文档片段通过向量模型转换成向量，然后存入向量库

## 调用阶段
4. 编写prompt将用户问题和检索到的文档片段拼接到一起
5. 将向量检索过程和模型调用串联起来

## 1 准备待检索的文档
我们以千帆平台上的预置服务收费文档为例进行后续的演示，这个文档可以见链接 https://cloud.baidu.com/doc/WENXINWORKSHOP/s/hlrk4akp7 。
我们把这个文档复制成txt文件，然后用langchain的TextLoader进行加载。

In [35]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/招商银行介绍.txt", encoding='utf8')
loader.load()

docs = loader.load()

In [36]:
docs

[Document(page_content='招商银行（China Merchants Bank）1987年成立于深圳蛇口，资产总额110285.00亿元， [90]是中国境内第一家完全由企业法人持股的股份制商业银行，也是国家从体制外推动银行业改革的第一家试点银行。 [1-3] [74]\n自2012年首次入围《财富》世界500强以来，已连续7年入围，2018年《财富》世界500强居213位。 [4-5]2019年6月26日，招商银行等8家银行首批上线运行企业信息联网核查系统。 [6]2019年7月，发布2019《财富》世界500强位列188位。 [7]2019年11月，“一带一路”中国企业100强榜单排名第85位。 [8]2019年12月，招商银行入选2019中国品牌强国盛典榜样100品牌。 [9]2019年12月18日，人民日报“中国品牌发展指数”100榜单排名第31位。 [10]2020年1月13日，入选2020胡润至尚优品获奖名单。2020年3月，入选2020年全球品牌价值500强第74位。 [11]2021年5月，位列“2021福布斯全球企业2000强”第22位。 [53]\nhttp://www.cmbchina.com/\n招商银行总行设在中国广东省深圳市福田区，2002年4月9日，招商银行A股在上海证券交易所挂牌上市。2006年9月8日，招商银行开始在香港公开招股，发行约22亿股H股，集资200亿港元，并在9月22日于港交所上市。资本净额超过2900亿、资产总额超过4.4万亿。 [3] [13-14]截至2017年上半年，招商银行境内外分支机构逾1800家，在中国大陆的130余个城市设立了服务网点，拥有5家境外分行和3家境外代表处，员工7万余人。此外，招商银行还在境内全资拥有招银金融租赁有限公司，控股招商基金管理有限公司，持有招商信诺人寿保险有限公司50%股权、招联消费金融公司50%股权；在香港全资控股永隆银行有限公司和招银国际金融控股有限公司。 [3]\n招商银行发展目标是成为中国领先的零售银行。1995年7月推出银行卡一卡通，1999年9月启动中国首家网上银行一网通，成为众多企业和电子商务网站广泛使用网上支付工具，在一定程度上促进了中国电子商务的发展。 [15-16]2015年4月2日，招商局集团有限公司筹划重大事项，为保证公平信息披露，维护投

## 2 将长文档切成小的文本片段
一般参考的文档都比较长，我们需要将他们切分成小的文本片段，然后将这些片段转换成向量，存入向量库留待以后检索用。文本片段是做检索的最小单位。

RecursiveCharacterTextSplitter可以做切分文档的工作，其中chunk_size表示切分片段的长度，chunk_overlap表示不同片段之间重叠的长度，separators表示切分时优先使用哪些符号作为片段之间的分割。

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0, separators=["\n\n", "\n", "。", "]", " ", ""])
documents = text_splitter.split_documents(docs)

In [38]:
print(len(documents))
documents

54


[Document(page_content='招商银行（China Merchants Bank）1987年成立于深圳蛇口，资产总额110285.00亿元， [90]是中国境内第一家完全由企业法人持股的股份制商业银行，也是国家从体制外推动银行业改革的第一家试点银行。 [1-3] [74]', metadata={'source': 'data/招商银行介绍.txt'}),
 Document(page_content='自2012年首次入围《财富》世界500强以来，已连续7年入围，2018年《财富》世界500强居213位。 [4-5]2019年6月26日，招商银行等8家银行首批上线运行企业信息联网核查系统。 [6]2019年7月，发布2019《财富》世界500强位列188位。 [7]2019年11月，“一带一路”中国企业100强榜单排名第85位', metadata={'source': 'data/招商银行介绍.txt'}),
 Document(page_content='。 [8]2019年12月，招商银行入选2019中国品牌强国盛典榜样100品牌。 [9]2019年12月18日，人民日报“中国品牌发展指数”100榜单排名第31位。 [10]2020年1月13日，入选2020胡润至尚优品获奖名单。2020年3月，入选2020年全球品牌价值500强第74位。 [11]2021年5月，位列“2021福布斯全球企业2000强”第22位。 [53]', metadata={'source': 'data/招商银行介绍.txt'}),
 Document(page_content='http://www.cmbchina.com/', metadata={'source': 'data/招商银行介绍.txt'}),
 Document(page_content='招商银行总行设在中国广东省深圳市福田区，2002年4月9日，招商银行A股在上海证券交易所挂牌上市。2006年9月8日，招商银行开始在香港公开招股，发行约22亿股H股，集资200亿港元，并在9月22日于港交所上市。资本净额超过2900亿、资产总额超过4.4万亿', metadata={'source': 'data/招商银行介绍.txt'}),
 Document(page_content='。 [3] [13

## 3.1 向量模型
将文档片段转换成向量需要向量模型，在千帆平台上同样提供了向量模型的调用服务，详见 https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Nlks5zkzu#%E5%90%91%E9%87%8Fembeddings
    

In [39]:
from langchain.embeddings import QianfanEmbeddingsEndpoint

embeddings = QianfanEmbeddingsEndpoint(model="Embedding-V1")  

## 3.2 向量库
文档片段转换成向量之后，需要存储到向量库中。这样在用户提出问题时，可以将问题文本经过向量模型提取问题向量，再用该问题向量检索向量库中最相似的文本片段。

这里我们使用facebook开源的一个向量库faiss。

In [40]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(documents, embeddings)

## 4 准备RAG的调用流程：拼接问题和检索片段
在用户问题检索到相似的文档片段时，我们需要考虑如何将这些文档片段和用户问题拼接在一起，输入给大模型。

这里用到了刚才提到的PromptTemplate，并构造了一个新的调用链 doucument_chain，来完成这个工作。

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_template = ChatPromptTemplate.from_template("""基于提供的【相关材料】回答最后的问题:
## 【相关材料】
{context}

## 最后的问题
问题: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt_template)

我们可以人工构造检索到的文档片段，然后用上面的doucument_chain进行大模型调用。这样来模拟检索的过程，测试我们的prompt模版。

In [42]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "招商银行在2019年获得了什么奖项?",
    "context": [Document(page_content='2021-12\tEDGE AWARDS 年度ESG实力先锋企业\t获奖\t[58]\n获奖时间\t奖项名称\t获奖结果\n2020\t2020胡润至尚优品\t获奖\n2019\t最佳股份制商业银行 [50]\t获奖\n2019\t最具投资价值银行 [50]\t获奖\n2019\t最佳手机银行 [50]\t获奖\n2014\t最具企业社会责任感公司奖\t获奖\n2014\t年度最佳财富管理银行\t获奖\n2014\t年度卓越私人银行服务\t获奖', metadata={'source': 'data/招商银行介绍.txt'})]
})

'根据提供的相关材料，招商银行在2019年获得了以下奖项：\n\n* 最佳股份制商业银行\n* 最具投资价值银行\n* 最佳手机银行\n\n这些奖项表明招商银行在当年在多个领域都取得了卓越的成就和认可。'

## 5 串联向量检索和大模型调用

测试模版没问题后，我们来组装调用流程的最后一个环节：向量检索。

这里在上面doucument_chain的基础上创建了一个新的调用链 retrieval_chain，来将向量库检索的过程加入到调用流程中。

In [43]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [44]:
response = retrieval_chain.invoke({"input": "招商银行在2014年获得了什么奖项?"})
print(response["answer"])

招商银行在2014年获得了以下两个奖项：

1. 最佳创新银行奖
2. 最佳中资银行奖

这两个奖项都是对招商银行在金融领域的卓越表现和创新的认可。


这样就完成了整个的RAG准备，RAG调用的流程！

另外，在reponse中可以查看context的字段，这就是问题在向量库中检索到的文档片段，这在调试RAG应用时很有用。

In [45]:
response

{'input': '招商银行在2014年获得了什么奖项?',
 'context': [Document(page_content='1999\t招商银行官网中国十大优秀网站之一\t获奖\n1999\t亚洲最大100家银行第一位\t获奖', metadata={'source': 'data/招商银行介绍.txt'}),
  Document(page_content='。2016年8月，招商银行在"2016中国企业500强"中排名第39位。 [17]', metadata={'source': 'data/招商银行介绍.txt'}),
  Document(page_content='2014\t最佳创新银行奖\t获奖\n2014\t最佳中资银行奖\t获奖\n2013\t深圳市金融创新奖\t获奖\n2013\t卓越手机银行奖\t获奖\n2013\t中国银行业星级服务机构\t获奖\n2013\t北亚最佳私人银行\t获奖\n2013\t年度广告主品牌奖\t获奖\n2012\t中国高端私人理财金鼎奖\t获奖\n2012\t年度十大品牌银行奖\t获奖\n2012\t品牌中国华谱奖\t获奖\n2012\t搜狐金融德胜奖年度企业\t获奖', metadata={'source': 'data/招商银行介绍.txt'}),
  Document(page_content='招商银行（China Merchants Bank）1987年成立于深圳蛇口，资产总额110285.00亿元， [90]是中国境内第一家完全由企业法人持股的股份制商业银行，也是国家从体制外推动银行业改革的第一家试点银行。 [1-3] [74]', metadata={'source': 'data/招商银行介绍.txt'})],
 'answer': '招商银行在2014年获得了以下两个奖项：\n\n1. 最佳创新银行奖\n2. 最佳中资银行奖\n\n这两个奖项都是对招商银行在金融领域的卓越表现和创新的认可。'}